## Semester 1 Project Submission

Please fill out:
* Student name: 
* Instructor name: 


In [1]:
# Your code here - remember to use markdown cells for comments as well!

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats

conn = sqlite3.connect('zippedData/im.db')

# Dylan Orndorf-Ronk

# Alex Bonczkiewicz

# Alberto Torres

# Travis McCloughan

# Jaemin Lee

In [3]:
q = """
SELECT name 
FROM sqlite_master 
WHERE type='table';
"""
pd.read_sql(q, conn)

,name
